In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import gensim
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')


### Loading the dataset

In [2]:
df = pd.read_csv('/home/einstein/Portfolio/Systeme_recommandations/data/spotify_songs.csv')
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [3]:
df.isnull().sum(), df.duplicated().sum()

(track_id                    0
 track_name                  5
 track_artist                5
 track_popularity            0
 track_album_id              0
 track_album_name            5
 track_album_release_date    0
 playlist_name               0
 playlist_id                 0
 playlist_genre              0
 playlist_subgenre           0
 danceability                0
 energy                      0
 key                         0
 loudness                    0
 mode                        0
 speechiness                 0
 acousticness                0
 instrumentalness            0
 liveness                    0
 valence                     0
 tempo                       0
 duration_ms                 0
 dtype: int64,
 0)

In [4]:
data = df[['track_name', 'track_artist', 'track_album_name', 'playlist_name']]
data.sample(5)

,track_name,track_artist,track_album_name,playlist_name
20827,Moni Mi Suku,Broederliefde,Broeders (Extended),🔥🦁 GOOD VIBES ONLY 🦁🔥 // BROEDERLIEFDE || FREN...
31664,Changes,Imagination,In the Heat of the Night,House/Electro/Progressive/Disco/Lofi/Synthwave
17408,I Got U - Radio Edit,Duke Dumont,Ibiza Residence 2014,EDM TROPICAL
18995,Saoco,Wisin,El Sobreviviente,Reggaeton Classics
20716,Attention,Charlie Puth,Attention,Today's Hits 2000-Present


In [5]:
data = data.dropna(axis = 0, how = 'any')

In [6]:
data.drop_duplicates()

,track_name,track_artist,track_album_name,playlist_name
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix
1,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),Pop Remix
2,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),Pop Remix
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,Pop Remix
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,Someone You Loved (Future Humans Remix),Pop Remix
...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,City Of Lights (Vocal Mix),♥ EDM LOVE 2020
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,Closer Remixed,♥ EDM LOVE 2020
32830,Sweet Surrender - Radio Edit,Starkillers,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020
32831,Only For You - Maor Levi Remix,Mat Zo,Only For You (Remixes),♥ EDM LOVE 2020


In [7]:
### renommer les colonnes
data = data.rename(columns = {"track_name": "Nom_chanson", 
                              "track_artist": "Artist_inter",
                              "track_album_name": "Nom_album",
                              'playlist_name': "Nom_playlist"})
data.head()

,Nom_chanson,Artist_inter,Nom_album,Nom_playlist
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix
1,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),Pop Remix
2,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),Pop Remix
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,Pop Remix
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,Someone You Loved (Future Humans Remix),Pop Remix


L'objectif est de mettre en place une application de recommandations des albumns et leurs chansons 
Ce qui veut quand l'utilisateur entre le nom de son artiste préféré, il obtient 05 autres artistes similaires suivis de leurs chansons et leurs album

In [28]:
### combinons les variables nécéssaires
data['Track'] = data['Artist_inter'] + " a fait sortir la chanson " + data["Nom_chanson"] + " et son album est " + data["Nom_album"]
data['Blood'] =  data['Artist_inter'] + " " + data["Nom_chanson"] + " " + data["Nom_album"]

In [29]:
data['Track'].head(1)

0    Ed Sheeran a fait sortir la chanson I Don't Ca...
Name: Track, dtype: object

In [30]:
data.isnull().sum(), data.duplicated().sum()

(Nom_chanson     0
 Artist_inter    0
 Nom_album       0
 Nom_playlist    0
 Track           0
 Blood           0
 dtype: int64,
 606)

In [31]:
data.drop_duplicates()

,Nom_chanson,Artist_inter,Nom_album,Nom_playlist,Track,Blood
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,Ed Sheeran a fait sortir la chanson I Don't Ca...,Ed Sheeran I Don't Care (with Justin Bieber) -...
1,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),Pop Remix,Maroon 5 a fait sortir la chanson Memories - D...,Maroon 5 Memories - Dillon Francis Remix Memor...
2,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),Pop Remix,Zara Larsson a fait sortir la chanson All the ...,Zara Larsson All the Time - Don Diablo Remix A...
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,Pop Remix,The Chainsmokers a fait sortir la chanson Call...,The Chainsmokers Call You Mine - Keanu Silva R...
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,Someone You Loved (Future Humans Remix),Pop Remix,Lewis Capaldi a fait sortir la chanson Someone...,Lewis Capaldi Someone You Loved - Future Human...
...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,Lush & Simon a fait sortir la chanson City Of ...,Lush & Simon City Of Lights - Official Radio E...
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,Closer Remixed,♥ EDM LOVE 2020,Tegan and Sara a fait sortir la chanson Closer...,Tegan and Sara Closer - Sultan & Ned Shepard R...
32830,Sweet Surrender - Radio Edit,Starkillers,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,Starkillers a fait sortir la chanson Sweet Sur...,Starkillers Sweet Surrender - Radio Edit Sweet...
32831,Only For You - Maor Levi Remix,Mat Zo,Only For You (Remixes),♥ EDM LOVE 2020,Mat Zo a fait sortir la chanson Only For You -...,Mat Zo Only For You - Maor Levi Remix Only For...


In [33]:
song = data['Blood'].apply(lambda x: x.split()).tolist()

In [34]:

model = Word2Vec(
    sentences= song,
    vector_size=200,
    window=5,
    min_count=2,
    workers=4
)

model.build_vocab(song)

In [35]:
len(model.wv.index_to_key)

19451

In [36]:
data.to_csv('/home/einstein/Portfolio/Systeme_recommandations/data/song.csv', index = False)

In [37]:
## enregister le models
model.save('recommandations.model')
